Created on Fri Jan 17 17:03:35 2019  
Group 9  
@authors:  

# Make the transition matrix from the database of events

In [1]:
import pandas as pd
import numpy as np
import json
from gensim.models import KeyedVectors

In [2]:
%run constants.ipynb
%run dictionnaries.ipynb
%run functions.ipynb

In [3]:
model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)

##### make the database

In [5]:
if True:
    prevoc = get_prevoc()
    voc = get_voc(model_w2v, prevoc)
    bdd = make_bdd(prevoc, 50)

bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)
bdd.tail()

,session,date,json,hash
259,49,28/04/2010 23h50m42s,"{""tab"": ""Seat"", ""filters"": {""manufacturer"": [""...","{""filters"": {""aircraft"": [""d_150_mascaret""], ""..."
260,49,28/01/2016 22h25m07s,"{""tab"": ""Seat"", ""filters"": {""manufacturer"": [""...","{""filters"": {""aircraft"": [""f_22"", ""il_114"", ""s..."
261,49,25/06/2016 20h42m02s,"{""tab"": ""Flight characteristics"", ""filters"": {...","{""filters"": {""aircraft"": [""bt_67_turbo_67"", ""d..."
262,49,22/01/2011 06h07m56s,"{""tab"": ""Flight characteristics"", ""filters"": {...","{""filters"": {""aircraft"": [""ba_609"", ""mig_27_ba..."
263,49,18/08/2015 05h08m34s,"{""tab"": ""Flight characteristics"", ""filters"": {...","{""filters"": {""aircraft"": [""fa_200_aero_subaru""..."


##### or load an existing one

In [6]:
bdd = pd.read_csv(bdd_directory+'df_bdd.csv', sep=',', engine='python',
                  index_col=0, encoding='utf-8').reset_index()

bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)
bdd.tail()

,session,date,json,hash
5,0,07/11/2016 00h25m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a350-900""], ""catego..."
6,0,07/11/2016 00h28m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a320""], ""category"":..."
7,0,07/11/2016 00h30m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a320""], ""category"":..."
8,0,07/11/2016 00h35m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a320""], ""category"":..."
9,0,07/11/2016 00h40m36s,"{""session"": 0, ""filters"": {""manufacturer"": [""A...","{""filters"": {""aircraft"": [], ""category"": [], ""..."


##### extract the states

In [7]:
etats = dict([(a, b)
              for (b, a) in enumerate(sorted(bdd[CT_bdd_hash].unique()))])
id_to_etats = dict((b, a) for (a, b) in etats.items())

##### extract the transitions

In [8]:
transitions = np.zeros([len(etats), len(etats)])
for session in bdd["session"].unique():
    for i in bdd[bdd["session"] == session].index[1:]:
        transitions[etats[bdd[CT_bdd_hash][i-1]],
                    etats[bdd[CT_bdd_hash][i]]] += 1

# to proba matrix
transitions = (transitions.T / np.sum(transitions, axis=1)).T
df_transitions = pd.DataFrame(transitions, columns=etats, index=etats)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


##### make a prediction

In [9]:
a = id_to_etats[0]

predict_next_state(a, df_transitions)

'{"filters": {"aircraft": ["a350-900"], "category": [], "company": [], "country": ["england", "france", "germany", "italy", "spain"], "date": ["01092017", "30092017"], "manufacturer": ["airbus"]}, "tab": "general"}'

In [10]:
df_transitions.to_csv(bdd_directory+'df_transitions.csv',
                      sep='§', encoding='utf-8')